SECTION 1: THE AGENT RUNTIME (`src/agents/`, ~300 files)

This is where agents actually THINK AND ACT - the consciousness loop.


THE CORE LOOP
The agent loop is NOT a TRADITIONAL `while(true)` loop. It's EVENT-DRIVEN AND
REQUEST-RESPONSE BASED, managed by the `pi-agent-core` SDK.


ENTRY POINT: `runEmbeddedPiAgent()` in `src/agents/pi-embedded-runner/run.ts:137`

The flow for every single turn:



   User Messages Arrives
   > runEmbeddedPiAgent - L137
      > Resolve workspace & session
      > Load/validate model config
      > resolve auth profiles with fallback
      > Enter retry loop
         > runEmbeddedAttempt - L399
            > Load session file from disk
            > Create all tools
            > Build system prompt
            > Create agent session object
            > Apply system prompt override
            > Subscribe to streaming events
            > Call SDK streaming to LLM
               > Events stream back:
                  > message_start
                  > message_update
                  > message_end
                  > tool_execution_start
                  > tool_execution_update
                  > tool_execution_end
                  > agent_end
            > Accumulate response
      > Handle errors  (L 474-797)
      > Return or retry   

Response Payload(s) sent back





THE "THINKING" -- It's Delegated

The actual LLM thinking is NOT in OpenClaw code. It's delegated to the Anthopic
SDK (or equivalent provider). What OpenClaw does it:
   1. Sent message to LLM with tools and system prompt
   2. LLM processes, decides what to do, generates response
   3. SDK streams back events
   4. OpenClaw subscribes to events and handles them




The event handlers live in `pi-embedded-subscribe.handlers.ts` (L 24-61), which
dispatches to:
   - `handleMessageStart/Update/End` (in `handlers.messages.ts`)
   - `handleToolExecutionStart/Update/End` (in `handlers.tools.ts`)
   - `handleAgentStart/End` (in `handlers.lifecycle.ts`)





MULTI-LEVEL FAILOVER (3 LEVELS DEEP)
   The retry loop at L392 handles failures gracefully::

   LEVEL 1 - Auth Profile Rotation:
   - If current API key/profile fails, try the next one for same model
   - Profiles in cooldown are skipped

   LEVEL 2 - Model Fallback:
   - When the model itself fails (not just auth), switch to fallback model
   - Configured via `config.agents.defaults.model.fallbacks[]`
   - Throws `FailoverError` to trigger model change

   LEVEL 3 - CONTEXT REDUCTION :
   - On context overflow: auto-compaction (summarise old messages)
   - Up to 3 compaction retries
   - If still too big: truncate oversized tool results

Error classification (`pi-embeddded-helpers.ts:30`): detects "auth", 
"rate_limit", "billing", "timeout", "unknown"




SYSTEM PROMPT CONSTRUCTION
`buildAgentSystemPrompt()` in `system-prompt.ts:164` builds the prompt with 
25+ SECTIONS in order:
   
   1. Identity - "You are a personal assistant running inside OpenClaw"#
   2. Tooling - Tool listing and descriptions
   3. Tool Call Style - Narration Guidance
   4. Safety - Self-preservation, manipulation safeguards
   5. CLI Quick Reference
   6. Skills - Conditional skill injection
   7. Memory Recall - If memory tools available
   8. Self-Update - For gateway tool
   9. Model Aliases - If configured
   10. Workspace - Working directory declaration
   11. Documentation
   12. Sandbox Info - If sandboxed
   13. User Identity
   14. Time/Timezone
   15. Workspace Files
   16. Reply Tags
   17. Messaging context
   18. Voice/TTS
   19. Group Chat Context
   20. Reactions - If enabled
   21. Reasoning Format
   22. Project Context - SOUL.md + context files
   23. Silent Replies
   24. Heartbeats
   25. Runtime - Agent ID, host, OS, model, shell, channel, capabilities


Prompt Modes: "full" (main agent), "minimal" (subagents - fewer sections), 
"none" (identity only)

In [ ]:
+-------------------------------------+
|                 ___                 |
|    __.--/)  .-~~   ~~>>>>>>>>   .-. |
|   (._\~  \ (        ~~>>>>>>>>.~.-' |
|     -~}   \_~-,    )~~>>>>>>>' /    |
|       {     ~/    /~~~~~~. _.-~     |
|        ~.(   `--~~/      /~ ~.      |
|   .--~~~~_\  \--~(   -.-~~-.  \     |
|   ```-'~~ /  /    ~-.  \ .--~ /     |
|        (((_.'    (((__.' ```-'      |
+-------------------------------------+

SUBAGENT SPAWNING

Via the `session_spawn` tool (`tools/sessions-spwan-tool.ts:83`):


> Main Agent calls sessions_spawn
   > Validate agent ID against allowlist 
   > Create child session key: "agent:{id}:subagent:{uuid}"
   > Resolve model (per-agent config > global default)
   > Build subagent system prompt (minimal mode)
   > Register & queue async execution 
> Main Agent continues without waiting
> Subagent runs indepndently via same `runEmbeddedPiAgent()` loop
> Main agent queries via: sessions_list, sessions_history, sessions_send

Key: subagents are NOT blocking. They run async. The parent can keep working.

SESSION STATE PERSISTENCE

- FORMAT: JSONL at `{workspace}/.openclaw/session.jsonl`
- CONTENT: Session header + user messages + assistant messages + tool calls/results
- REOPENED at each run - previous messages loaded into memory
- COMPACTION: When context window exceeded, old messages get summarised into a
  single "summary" message




AGENT CONFIGURATION LAYERING

4 levels, each cascading:
   1. Global defaults (`config.agents/defaults`)
   2. Per-agent overrides (`config.agents.list[].model`, `.workspace`, etc.)
   3. Runtime overrides (sessionKey agent prefix)
   4. Subagent overrides

SECTION 2: THE TOOL SYSTEM

This is how agents EXECUTE ACTIONS in the real world.


WHERE TOOLS ARE DEFINED
   > `src/agents/pi-tools.ts:15`
      > Main tool creation: `createOpenClawCodingTools()`
   > `src/agents/openclaw-tools.ts:22`
      > OpenClaw-specific tools: `createOpenClawTools()`
   > `src/agents/tools/`
      > Individual tool implementations (22+ files)
   > `src/agents/bash-tools.exec.ts:800`
      > Full bash exec logic: `createExecTool()`
   > `src/agents/bash-tools.process.ts:44`
      > Background process management
   > `src/plugins/tools.ts:43`
      > Plugin tool registration               

TOOL INTERFACE

Every tool implements:
```ts
AgentTool<ParametersType, DetailsType> {
    name: string;
    label: string;
    description: string;
    parameters: JSONSchema;
    execute: async (
        toolCallId: string,
        args: unknown,
        signal?: AbortSignal,
        onUpdate?: (partialResult: AgentToolResult) => void
    ) => AgentToolResult<DetailType>
}
```

Result structure:
```ts
AgentToolResult {
    content: Array<{
        type: "text" | "image" | "audio"
        text?: string
        data?: string       // base 64
        mimeType?: string
    }>,
    details?: DetailsType       // Tool-specific structured data
}
```

Created in `openclaw-tools.ts:22`:

1. `browser` - Web browser control (Playwright)
2. `canvas` - Present/eval/snapshot canvas UI
3. `nodes` - List/describe/notify/camera/screen on paired devices
4. `cron` - Schedule wake events and cron jobs
5. `message` - Send messages and channel actions (conditional)
6. `tts` - Text-to-speech
7. `gateway` - Restart/config/update OpenClaw itself

In [ ]:
                            ...,ooooooooo......
                      .o8888888888888888888888888o.
                  .o888888888888888888888888888888888o.
                o8888888888A88"V888888888888888888888888o
              o88888887"8"  "   V888  88888888888888888888o
            o88888888            V     888888888888888888888o
           o888888888                   888888888888888888888o
          .88888888888                  88888V"  "V88888888888.
          o88888888888v                 8888"  v8  88888888888o
          88888888888v                  8888v  v88 888888888888
          888888888888                  88888v  "88888888888888
           88888888888V                  V88888v  "88888888888
           88888888888v                            "8888888888
____________8888888888888v.........................v888888888_____________
:::::::::::::::::::::::::'                         :::::::::::::::::::::::
:::::::::::::::::::::::                .:::::::    .::::::::::::::::::::::
::::::::::::::::::::::                 :::::::  .:::::::::::::::::::::::::
:::::::::::::::::::::                  ::::::  ::: :::::::::::::::::::::::
:::::::::::::::::::::.                 ::::::. :: .:::::::::::::::::::::::
::::::::::::::::::::::                 :::::::.  .::::::::::::::::::::::::
:::::::::::::::::::::.           .     :::::::::::::::::::::::::::::::::::
:::::::::::::::::::::          :::.   ::::::::::::::::::::::::::::::::::::
::::::::::::::::::::::.::.:: :::::::.:::::::::::::::::::::::::::::::::::::
::::::::::::::::::::::::::::::::::::::::::::::::::::Meindert+Stewart::::::
[Nice job.]

---

- Pi-mono (`badlogic/pi-mono`) is an FOSS TypeScript monorepo that serves as a
  toolkit for building, managing, and deploying AI agents. It is heavily 
  assosciatied with Pi, a minimal terminal-based coding agent designed by Mario
  to be a lean, extensible alternative to more complex AI coding tools.

- In the context of PI-MOO (a TS monorepo for an AI coding agent), "streaming
  agents" refer to the core functionality of the AI, which incrementally 
  processes data, generates text, execute tools., and updates the Terminal UI
  (TUI) in real-time. The agent utilises TS, npm workspaces, and support 
  multi-LLM provider LLMs.

  1. `pi-agent-core` (The Core Engine)
     - AGENT LOOP: Handles the main loop of the AI agent, including tool 
       execution, validation, and event streaming.
     - AgentMessage: The agent operates with a flexible `AgentMessage` type
       that supports standard LLM messages (user, assistant, tool result) and
       custom app-specific types.
     - STREAMING BEHAVIOR: When the agent is streaming, it manages how messages
       are handled, requiring `streamingBehavior` (such as `steer` or `followUp`)
       if a command is sent during an active stream.

  2. `pi-tui` (STREAMING OUTPUT)
     - REAL-TIME Interaction: The Terminal UI (TUI) framework provides "almost"
       flicker-free updates, rendering streaming text, tool output, and 
       markdown in real-time.
     - Differential Rendering: The TUI updates only the necessary parts of the
       screen, providing a smooth user experience while the model streams output
                  
  


  - ... "subscribing to events" refers to an event-driven, rather than 
    poll-based, architecture where the agent actively listens for specific, 
    typed signals (WebSocket frames) from the GateWay.

    This mechanism allows the system to react instantly to changes rather than
    constantly asking "what's new?".

    Here is what "subscribe to event" specifically means for OpenClaw:
       - REAL-TIME RESPONSIVENESS: Instead of polling, the agent is notified
         immediately when specific events occur, such as incoming messages from
         connected apps (WhatsApp, Telegram, Slack), or when internal state 
         changes.
       - EVENT TYPES: Agents can subscribe to various event types handled by the
         Gateway, including `agent`, `presence`, `health` and `tick` (time-based
         heartbeats).
       - AUTOMATED TRIGGERS (Heartbeat): The "heartbeat" feature is a crucial
         subscribe event where the agent triggers a check, acts on information
         (like a new email or scheduled task), and then returns to an idle state
         , functioning as a 24/7 personal assistant.
       - STRUCTURED DATA INTERACTION: These subscribed events are typed and 
         validated against Json Schema (generated from TypeBox), ensuring that
         the agent only processes well-formed data.
       - PROACTIVE CAPABILITIES: Subscribing to events allows OpenClaw to move
         beyond simple chat, enabling proactive "push" actions, such as sending
         a daily briefing via a Telegram channel without being prompted.       
    
    Essentially, subscribing to events enables OpenClaw to operate as a 24/7
    automated agent that triggers actions based on external triggers 
    (messages/alerts) rather than waiting for user input.



- Handlers in the backend are functions or classes designed to process incoming
  requests (HTTP, events, etc.), execute buisness logic, and return a response.
  They act as intermediaries between raw ssytem events or API endpoints and the
  core application logic, often managing data retrieval, validation and security
  tasks.

  Key aspects of handlers in backend development:
  - HTTP REQUEST HANDLING: Web handlers receive a request object (URLs, headers)
    and a response object, filling the latter with JSON, HTML or status codes.
  - EVENT HANDLING: In event-driven architectures, handlers (or consumers) 
    respond to specific events, such as processing data from a queue or a 
    database change.
  - PROCESS FLOW: A handler typically follows a structured path: 
     receive request --> validate data --> interact with db/models --> return response
  - TYPES OF HANDLERS:
      - PIPE HANDLERS: Common in monitoring systems (like Sensu) to process
        incoming events via scripts.
      - WEB SERVICE HANDLERS: Used to handle SOAP or REST API requests, often
        managing headers and endpoints.
      - DEFAULT HANDLERS: Built-in server functions that handle static content
        or default file requests.
  
  In short, a handler is a specialised function that "handles" a particular
  input to ensure the application reacts correctly to external requests or 
  system events.
  



- A cron job is a scheduled task in Unix-like operating system that 
  automatically runs commands or scripts at specififed intervals, such as daily,
  hourly, or weekly. Managed by the `cron daemon` (background process) via a 
  `crontab` (configuration file), they are widely used for automating system
  maintenance, backups, and repetitive tasks.  


- Gateway: Because everything runs through one process, the Gateway is a single
           control surface. Which model to call, which tools to allow, how much
           context to include, how much autonomy to grant -- all configured in 
           one place.  